<a href="https://colab.research.google.com/github/sonagidevelop/HonGongMachine/blob/main/Bookcode/5.%20%ED%8A%B8%EB%A6%AC%20%EC%95%8C%EA%B3%A0%EB%A6%AC%EC%A6%98/5_2_%EA%B5%90%EC%B0%A8_%EA%B2%80%EC%A6%9D%EA%B3%BC_%EA%B7%B8%EB%A6%AC%EB%93%9C_%EC%84%9C%EC%B9%98-2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 5-2. 교차 검증과 그리드 서치

**검증 세트**validation set   
테스트 세트로 지속적인 테스트를 하고 오차를 줄일 경우, 테스트 세트에 점점 맞추는 꼴이 된다. 따라서 가능한 한 테스트 세트를 사용하지 않는 것이 좋다.

In [1]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine-date')

In [2]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [3]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state = 42)

sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

In [4]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [5]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state = 42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


**교차 검증** cross validation   
**k-폴드 교차 검증** k-fold cross validation은 훈련세트를 k 부분으로 나누어 검증세트를 만드는 것이다. 그 여러개의 검증 세트의 점수의 평균을 구한다.

In [6]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.01097465, 0.00780129, 0.00808644, 0.00748706, 0.00706339]), 'score_time': array([0.00091958, 0.00077987, 0.00078773, 0.0007422 , 0.00068331]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [7]:
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


In [8]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv =StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [9]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv = splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


# 하이퍼파라미터 튜닝

In [11]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease' : [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

In [12]:
gs.fit(train_input, train_target)
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [16]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [17]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [18]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [19]:
params = {'min_impurity_decrease' : np.arange(0.0001, 0.001, 0.0001),
          'max_depth' : range(5, 20, 1),
          'min_samples_split' : range(2, 100, 10)}

In [20]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [21]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


---

In [22]:
from scipy.stats import uniform, randint

In [23]:
rgen = randint(0,10)
rgen.rvs(10)

array([7, 2, 4, 4, 6, 0, 1, 6, 3, 8])

In [24]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([102, 107, 102,  95, 110, 107,  84,  90, 115,  88]))

In [27]:
ugen = uniform(0,1)
ugen.rvs(10)

array([0.45938566, 0.08675076, 0.96474093, 0.15588298, 0.65930036,
       0.59718595, 0.984654  , 0.24277259, 0.35838698, 0.9393226 ])

In [28]:
params = {'min_impurity_decrease' : uniform(0.0001, 0.001),
          'max_depth' : randint(20, 50),
          'min_samples_split' : randint(2,25),
          'min_samples_leaf' : randint(1, 25),
          }

In [29]:
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter = 100 , n_jobs= -1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(cv=None, error_score=nan,
                   estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features=None,
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    presort='deprecated',
                                                    random_state=42,
         

In [30]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [31]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [32]:
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.86
